# env: swabian

In [66]:
import TimeTagger as TT
from TimeTagger import Flim, TimeTagStream, createTimeTaggerVirtual

In [32]:
fr = TT.FileReader(r"C:\Users\lociu\Desktop\PriyaM\Data\Swabian_tests\test.ttbin")

# flim

In [92]:
ttv=TT.createTimeTaggerVirtual()

RuntimeError: No license file found or no Time Tagger connected. Please connect a Time Tagger if you want to use the virtual Time Tagger for the first time.

In [81]:
laser = 1
click1 = -2
click2 = -3
frame = 4
pixel = 5
line = 6
bins = 100
binwidth = 125
xDim = 513
yDim = 513

laser_frequency = 80e6
total_frames = 2

In [91]:
flim1 = Flim(ttv, start_channel=laser, click_channel=click1, pixel_begin_channel=pixel, n_pixels=xDim*yDim,
             n_bins=bins, binwidth=binwidth, frame_begin_channel=frame, finish_after_outputframe=total_frames, n_frame_average=1)

TypeError: in method 'new_Flim', argument 2 of type 'TimeTaggerBase *'

# swab ex

In [34]:
# The format for the table and the head of the table
format_string = '{:>8} | {:>17} | {:>7} | {:>14} | {:>13}'
#print(format_string.format('TAG #', 'EVENT TYPE', 'CHANNEL', 'TIMESTAMP (ps)', 'MISSED EVENTS'))
#print('---------+-------------------+---------+----------------+--------------')

In [35]:
%%time

n_events = 100000  # Number of events to read at once
event_name = ['0 (TimeTag)', '1 (Error)', '2 (OverflowBegin)', '3 (OverflowEnd)', '4 (MissedEvents)']
i = 0

t_events =0
while fr.hasData():
    # getData() does not return timestamps, but an instance of TimeTagStreamBuffer
    # that contains more information than just the timestamp
    data = fr.getData(n_events=n_events)

    # With the following methods, we can retrieve a numpy array for the particular information:
    channel = data.getChannels()            # The channel numbers
    timestamps = data.getTimestamps()       # The timestamps in ps
    overflow_types = data.getEventTypes()   # TimeTag = 0, Error = 1, OverflowBegin = 2, OverflowEnd = 3, MissedEvents = 4
    missed_events = data.getMissedEvents()  # The numbers of missed events in case of overflow

    # Output to table
    if i < 2 or not fr.hasData():
        print(format_string.format(*" "*5))
        heading = ' Start of data chunk {} with {} events '.format(i+1, data.size)
        extra_width = 69 - len(heading)
        print('{} {} {}'.format("="*(extra_width//2), heading, "="*(extra_width - extra_width//2)))
        print(format_string.format(*" "*5))
        print(format_string.format(i*n_events + 1, event_name[overflow_types[0]], channel[0], timestamps[0], missed_events[0]))
        if data.size > 1:
            print(format_string.format(i*n_events + 2, event_name[overflow_types[1]], channel[1], timestamps[1], missed_events[1]))
        if data.size > 3:
            print(format_string.format(*["..."]*5))
        if data.size > 2:
            print(format_string.format(i*n_events + data.size, event_name[overflow_types[-1]], channel[-1], timestamps[-1], missed_events[-1]))
    if i == 1:
        print(format_string.format(*" "*5))
        for j in range(3):
            print(format_string.format(*"."*5))
    i += 1
    
    t_events = t_events + len(channel)
    

#show that it's done
print("done")


         |                   |         |                |              
=============  Start of data chunk 1 with 100000 events  ==============
         |                   |         |                |              
       1 |       0 (TimeTag) |       1 | 279827555654787 |             0
       2 |       0 (TimeTag) |      -1 | 279827555661643 |             0
     ... |               ... |     ... |            ... |           ...
  100000 |       0 (TimeTag) |       1 | 279828179922494 |             0
         |                   |         |                |              
=============  Start of data chunk 2 with 100000 events  ==============
         |                   |         |                |              
  100001 |       0 (TimeTag) |      -1 | 279828179929321 |             0
  100002 |       0 (TimeTag) |       1 | 279828179934930 |             0
     ... |               ... |     ... |            ... |           ...
  200000 |       0 (TimeTag) |       1 | 279828804139874 | 

#### length of events : 1854399131 

In [36]:
fr.getData(1).getChannels()

array([], dtype=int32)

# my exp:

In [37]:
## pretty print for json
fr.getConfiguration()

{'channel number scheme': 2,
 'configure version': 1,
 'current time': '2022-06-10 16:33:51 -0500',
 'devices': [{'PCB version': '1.6b (1)',
   'model': 'Time Tagger Ultra',
   'serial': '2143000XZI'}],
 'firmware version': 'TT-Ultra, FW 5, TS 2022-02-02 13:09:16, OK 1.31',
 'hardware buffer size': 67108864,
 'inputs': [{'channel': [1, -1],
   'conditional filter filtered': [False, False],
   'conditional filter triggers': [False, False],
   'deadtime': [2000, 2000],
   'delay hardware': [0, 0],
   'delay software': [0, 0],
   'event divider': [0, 0],
   'hardware delay compensation': [552, 552],
   'input mux': 0,
   'normalization': [True, True],
   'resolution': 'Standard',
   'resolution rms': 9.0,
   'trigger level': -0.050011444266422345},
  {'channel': [2, -2],
   'conditional filter filtered': [False, False],
   'conditional filter triggers': [False, False],
   'deadtime': [2000, 2000],
   'delay hardware': [0, 0],
   'delay software': [0, 0],
   'event divider': [0, 0],
   'ha

In [ ]:
buf = fr.getData(0)
print(buf.tStart, buf.getTimestamps(), buf.getChannels())

#### last byte on the stream : 329628425460000 


In [19]:
fr = TT.FileReader(r"C:\Users\lociu\Desktop\PriyaM\Data\Swabian_tests\test.ttbin")

In [39]:
fr.getChannelList()

(1, 2, 3, 4, -1, -2, -3, -4)

In [40]:
buf = fr.getData(0)
print(buf.tStart, buf.getTimestamps(), buf.getChannels())

-1 [] []


In [41]:
total_events

1854399131.0

In [38]:
n_events = int(1854399131.0)

while fr.hasData():
    data = fr.getData(n_events=n_events)

    # With the following methods, we can retrieve a numpy array for the particular information:
    channel = data.getChannels()            # The channel numbers
    timestamps = data.getTimestamps()       # The timestamps in ps
    overflow_types = data.getEventTypes()   # TimeTag = 0, Error = 1, OverflowBegin = 2, OverflowEnd = 3, MissedEvents = 4
    missed_events = data.getMissedEvents()  # The numbers of missed events in case of overflow
    

In [54]:
type(data)

TimeTagger.TimeTagStreamBuffer

In [39]:
len(channel)

1854399131

In [7]:
import numpy as np

In [49]:
%%time
len(np.where(channel==1))

CPU times: total: 5.92 s
Wall time: 6 s


1

In [57]:
%%time
for ch in range(-4,5):
    ch_1 = np.where(channel==ch)
    print(ch, ch_1[0].shape)
    

-4 (2378188,)
-3 (52,)
-2 (4863,)
-1 (924816815,)
0 (0,)
1 (924816812,)
2 (4854,)
3 (62,)
4 (2377401,)
CPU times: total: 15.3 s
Wall time: 15.3 s


In [ ]:
1:laser/sync
2: line clock
3: frame clock
4: photons

In [23]:
def tests(n,m):
    print("\nSize:")
    for i in range(n,m):
        x = fr.getData(i)
        print(i,":", x.size)
    
    print("\nChannels:")
    for i in range(n,m):
        x = fr.getData(i)
        print(i,":", x.getChannels())
        
    print("\nEvent Types:")    
    for i in range(n,m):
        x = fr.getData(i)
        print(i,":", x.getEventTypes())
        
    print("\nTime Stamps:")
    for i in range(n,m):
        x = fr.getData(i)
        print(i,":", x.getTimestamps())
        
    print("\nGet Data:")
    for i in range(n,m):
        x = fr.getData(i)
        print(i,":", x.tGetData)
        
    print("\nT Start:")
    for i in range(n,m):
        x = fr.getData(i)
        print(i,":", x.tStart)

In [24]:
tests(1,10)


Size:
1 : 0
2 : 0
3 : 0
4 : 0
5 : 0
6 : 0
7 : 0
8 : 0
9 : 0

Channels:
1 : []
2 : []
3 : []
4 : []
5 : []
6 : []
7 : []
8 : []
9 : []

Event Types:
1 : []
2 : []
3 : []
4 : []
5 : []
6 : []
7 : []
8 : []
9 : []

Time Stamps:
1 : []
2 : []
3 : []
4 : []
5 : []
6 : []
7 : []
8 : []
9 : []

Get Data:
1 : 329628425460000
2 : 329628425460000
3 : 329628425460000
4 : 329628425460000
5 : 329628425460000
6 : 329628425460000
7 : 329628425460000
8 : 329628425460000
9 : 329628425460000

T Start:
1 : 329628425460000
2 : 329628425460000
3 : 329628425460000
4 : 329628425460000
5 : 329628425460000
6 : 329628425460000
7 : 329628425460000
8 : 329628425460000
9 : 329628425460000


# server and client?